In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#DATA WRANGLING AND ANALYSING
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random as rnd

#DATA VISUALISATION
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#MACHINE LEARNING
from sklearn.svm import SVC , LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_path='../input/titanic-machine-learning-from-disaster/train.csv' #Importing train and test csv files
test_path='../input/titanic-machine-learning-from-disaster/test.csv'
train_df=pd.read_csv(train_path)
test_df=pd.read_csv(test_path)
combine=[train_df,test_df]

In [ ]:
train_df.head()
print(train_df.columns.values) # Columns names in train.csv file

In [ ]:
train_df.info()  # Information about csv files
test_df.info()

In [ ]:
train_df.describe()

In [ ]:
train_df.describe(include=['O'])


In [ ]:
train_df[['Pclass','Survived']].groupby(['Pclass'],as_index=False).mean().sort_values(by='Survived',
                                                                                      ascending=False)

In [ ]:
train_df[['Sex','Survived']].groupby(['Sex'],as_index=False).mean().sort_values(by='Survived',
                                                                               ascending=False)

In [ ]:
train_df[['SibSp','Survived']].groupby('SibSp').mean().sort_values(by='Survived',ascending=False)

In [ ]:
train_df[['Parch','Survived']].groupby(['Parch']).mean().sort_values(by='Survived',ascending=False)

In [ ]:
g=sns.FacetGrid(train_df,col='Survived')
g.map(plt.hist,'Age',bins=20)

In [ ]:
g=sns.FacetGrid(train_df,col='Survived',row='Pclass',height=2.2,aspect=1.6)
g.map(plt.hist,'Age',bins=20)

In [ ]:
grid=sns.FacetGrid(train_df,row='Embarked',height=2.2,aspect=1.6)
grid.map(sns.pointplot,'Pclass','Survived','Sex',palette='deep')
grid.add_legend()

In [ ]:
grid=sns.FacetGrid(train_df,row='Embarked',col='Survived',height=2.2,aspect=1.6)
grid.map(sns.barplot,'Sex','Fare',alpha=0.5,ci=None)
grid.add_legend()

In [ ]:
train_df=train_df.drop(['Ticket','Cabin'],axis=1)
test_df=test_df.drop(['Ticket','Cabin'],axis=1)
combine=[train_df,test_df]

In [ ]:
for dataset in combine:
    dataset['Title']=dataset.Name.str.extract('([A-Za-z]+)\.',expand=False)
    
pd.crosstab(train_df['Title'],train_df['Sex'])

In [ ]:
for dataset in combine:
    dataset['Title']=dataset['Title'].replace(['Lady','Countess','Capt','Col','Don','Dr','Major','Rev','Sir','Jonkheer','Dona'],'Rare')
    dataset['Title']=dataset['Title'].replace('Mlle','Miss')
    dataset['Title']=dataset['Title'].replace('Ms','Miss')
    dataset['Title']=dataset['Title'].replace('Mme','Mrs')
    

train_df[['Survived','Title']].groupby(['Title'],as_index=False).mean().sort_values(by='Survived',ascending=False)
#train_df.head()

In [ ]:
title_mapping={"Mr":1,"Miss":2,"Mrs":3,"Master":4,"Rare":5}
for dataset in combine:
    dataset['Title']=dataset['Title'].map(title_mapping)
    dataset['Title']=dataset['Title'].fillna(0)
    
train_df.head()

In [ ]:
train_df=train_df.drop(['Name','PassengerId'],axis=1)
test_df=test_df.drop(['Name'],axis=1)
combine=[train_df,test_df]


In [ ]:
for dataset in combine:
    dataset["Sex"]=dataset["Sex"].map({'female':0,'male':1}).astype(int)
train_df.head()

In [ ]:
grid=sns.FacetGrid(train_df,row='Pclass',col='Survived',height=2.2,aspect=1.6)
grid.map(plt.hist,'Age',alpha=0.5,bins=20)
grid.add_legend()

In [ ]:
guess_ages=np.zeros((2,3))
for dataset in combine:
    for i in range(0,2):
        for j in range(0,3):
            guess_df=dataset[(dataset['Sex']==i) & (dataset['Pclass']==j+1)]['Age'].dropna()
            
            age_guess=guess_df.median()
            
            guess_ages[i,j]=int(age_guess/0.5+0.5)*0.5
            
            
    for i in range(0,2):
        for j in range(0,3):
            dataset.loc[(dataset.Age.isnull()) & (dataset.Sex==i) & (dataset.Pclass==j+1),'Age']=guess_ages[i,j]
            
train_df.head()
            

In [ ]:
train_df['AgeBand']=pd.cut(train_df["Age"],5)
train_df[['Survived','AgeBand']].groupby('AgeBand',as_index='False').mean().sort_values(by='Survived',ascending=False)

In [ ]:
for dataset in combine:
    dataset.loc[dataset['Age']<=16,'Age']=0
    dataset.loc[(dataset['Age']>16) & (dataset['Age']<=32),'Age']=1
    dataset.loc[(dataset['Age']>16) & (dataset['Age']<=48),'Age']=2
    dataset.loc[(dataset['Age']>16) & (dataset['Age']<=64),'Age']=3
    dataset.loc[dataset['Age']>64,'Age']
    
train_df.head()
    
    

In [ ]:
train_df=train_df.drop("AgeBand",axis=1)
combine=[train_df,test_df]



In [ ]:
for dataset in combine:
    dataset['FamilySize']=dataset['Parch']+dataset['SibSp']+1
    
train_df[['Survived','FamilySize']].groupby('FamilySize').mean().sort_values(by='Survived',ascending=False)

In [ ]:
for dataset in combine:
    dataset['IsAlone']=0
    dataset.loc[dataset['FamilySize']==1 , 'IsAlone']=1
train_df[['Survived','IsAlone']].groupby('IsAlone').mean().sort_values(by='Survived',ascending=True)

In [ ]:
train_df=train_df.drop(['Parch','FamilySize','SibSp'],axis=1)
test_df=test_df.drop(['Parch','FamilySize','SibSp'],axis=1)
combine=[train_df,test_df]
train_df.head()

In [ ]:
#NEW FEATURE
for dataset in combine:
    dataset['Age*Class']=dataset['Age']*dataset['Pclass']
train_df.loc[:,['Age','Pclass','Age*Class']].head(10)

In [ ]:
freq_port=train_df.Embarked.dropna().mode()[0]
freq_port
for dataset in combine:
    dataset['Embarked']=dataset['Embarked'].fillna(freq_port)
    
for dataset in combine:
    dataset['Embarked']=dataset['Embarked'].map({'S':0,"C":1,"Q":2}).astype(int)
    
train_df.head()

In [ ]:
dataset['Fare'].fillna(dataset['Fare'].dropna().median(),inplace=True)
train_df.head()

train_df['FareBand']=pd.qcut(dataset['Fare'],4)

train_df[["Survived","FareBand"]].groupby('FareBand').mean().sort_values(by='Survived',ascending=True)

In [ ]:
for dataset in combine:
    dataset.loc[dataset['Fare']<=7.91,'Fare']=0
    dataset.loc[(dataset['Fare']>7.91) & (dataset['Fare']<=14.454),"Fare"]=1
    dataset.loc[(dataset['Fare']>14.454) & (dataset['Fare']<=31.472),"Fare"]=2
    dataset.loc[(dataset['Fare']>31.472) & (dataset['Fare']<=512.329),"Fare"]=3
    dataset['Fare']=dataset['Fare'].astype(int)
    
train_df=train_df.drop(['FareBand'],axis=1)
combine=[train_df,test_df]
train_df.head()

In [ ]:
X_train=train_df.drop("Survived",axis=1)
Y_train=train_df['Survived']
X_test=test_df.drop(["PassengerId"],axis=1).copy()


In [ ]:
#MODEL
#SUPPORT VECTOR MACHINE
svc=SVC()
svc.fit(X_train,Y_train)
y_predict=svc.predict(X_test)
acc_svc=round(svc.score(X_train,Y_train)*100,2)
acc_svc

In [ ]:
#KNN
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train,Y_train)
y_predict=knn.predict(X_test)
scc=round(knn.score(X_train,Y_train)*100,2)
scc

In [ ]:
random=RandomForestClassifier(n_estimators=100)
random.fit(X_train,Y_train)
y_predict=random.predict(X_test)
scc=round(random.score(X_train,Y_train)*100,2)
scc

In [ ]:
submission=pd.DataFrame({"PassengerId":test_df['PassengerId'],"Survived":y_predict})
#submission
submission.to_csv('submission.csv',index=False)
submission